In [ ]:
from satsearch import Search
import stackstac, os, requests
from netrc import netrc
from subprocess import Popen
from getpass import getpass

In [ ]:
#Setup NASA Credentials

urs = 'urs.earthdata.nasa.gov'    # Earthdata URL to call for authentication
prompts = ['Enter NASA Earthdata Login Username \n(or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']
homeDir = os.path.expanduser("~")
Popen('touch {0}.netrc | chmod og-rw {0}.netrc | echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
Popen('echo login {} >> {}.netrc'.format(getpass(prompt=prompts[0]), homeDir + os.sep), shell=True)
Popen('echo password {} >> {}.netrc'.format(getpass(prompt=prompts[1]), homeDir + os.sep), shell=True)
del homeDir, urs, prompts

In [ ]:
#Get NASA Temp AWS Credentials
s3_cred = requests.get('https://lpdaac.earthdata.nasa.gov/s3credentials').json()
s3_cred

In [ ]:
env = dict(GDAL_DISABLE_READDIR_ON_OPEN='YES', 
           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
           GDAL_SWATH_SIZE='200000000',
           VSI_CURL_CACHE_SIZE='200000000',
           CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
           GDAL_HTTP_UNSAFESSL='YES',
           GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
           GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'),
           AWS_REGION='us-west-2',
           AWS_SECRET_ACCESS_KEY=s3_cred['secretAccessKey'],
           AWS_ACCESS_KEY_ID=s3_cred['accessKeyId'],
           AWS_SESSION_TOKEN=s3_cred['sessionToken'])

In [ ]:
def get_STAC_items(url, collection, dates, bbox):
  results = Search.search(url=url,
                      collections=collection, 
                      datetime=dates,
                      bbox=bbox)

  return(results)

def remap_s3_url(stac):
    for i,entry in enumerate(s_col):
        for asset in entry['assets'].keys():
            s_col[i]['assets'][asset]['href'] = s_col[i]['assets'][asset]['href'].replace('https://lpdaac.earthdata.nasa.gov/',
                                                                                          '/vsis3/')
    return(stac)

url = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/' 
collection = ['HLSS30.v1.5']#'C1711924822-LPCLOUD' #HLS
bbox=[-104.79107047,   40.78311181, -104.67687336,   40.87008987]
dates = '2013-02-01/2021-04-10'

stac_items_hls = get_STAC_items(url,collection,dates,','.join(map(str, bbox))).items()
s_col_hls = stac_items_hls.geojson()['features']

s_col_hls_s3 = remap_s3_url(s_col_hls)

In [ ]:
url = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/' 
collection = ['HLSS30.v1.5']#'C1711924822-LPCLOUD' #HLS
bbox=[-104.79107047,   40.78311181, -104.67687336,   40.87008987]
dates = '2013-02-01/2021-04-10'

stac_items_hls = get_STAC_items(url,collection,dates,','.join(map(str, bbox))).items()
s_col_hls = stac_items_hls.geojson()['features']

s_col_hls_s3 = remap_s3_url(s_col_hls)

In [ ]:
stackstac.stack(s_col_hls,resolution=30,epsg=32613,assets=['B01'],gdal_env=stackstac.LayeredEnv(env))

In [ ]:
stackstac.LayeredEnv.always()